In [5]:
import os
import difflib
import collections
from datetime import datetime
import openpyxl

In [35]:
# enter basic informations like checkpoint_path and configs_path,
ckpt_path = "/root/ext_data/studiogan_ckpt/AFHQv2/checkpoints"
cfg_path = "/root/code/joong_test/PyTorch-StudioGAN/src/configs/AFHQv2"
dataset_path = "/root/ext_data/data/AFHQv2"
dataset = "AFHQv2"
num_eval = 1

ckpt_list = os.listdir(ckpt_path)
cfg_list = os.listdir(cfg_path)

# load Excel File and read informations
wb = openpyxl.load_workbook("Taxonomy_experiments_offline.xlsx")
ws = wb['Efficient-AFHQ-V2']

runs = collections.defaultdict(lambda : collections.defaultdict(str))

all_columns = ws.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        assert col[6].value in ckpt_list, str(col[6].value) + " Not in ckpt_list"
        runs[col[1].value]["run_1"] = col[6].value
        if col[7].value != None:
            assert col[7].value in ckpt_list, str(col[7].value) + " Not in ckpt_list"
            runs[col[1].value]["run_2"] = col[7].value
        if col[8].value != None:
            assert col[8].value in ckpt_list, str(col[8].value) + " Not in ckpt_list"
            runs[col[1].value]["run_3"] = col[8].value
            
        if col[1].value == "ACGAN":
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value+"-Mod", cfg_list, n=1, cutoff=0)[0]
        else:
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value, cfg_list, n=1, cutoff=0)[0]
        print(f"{col[1].value} matched with {runs[col[1].value]['cfg']}")

BigGAN matched with BigGAN-256Res.yaml
BigGAN-DiffAug matched with BigGAN-256Res-DiffAug.yaml
BigGAN-ADA matched with BigGAN-256Res-ADA.yaml
BigGAN-APA matched with BigGAN-256Res-APA.yaml
BigGAN-LeCam matched with BigGAN-256Res.yaml
BigGAN-DiffAug-LeCam matched with BigGAN-256Res-DiffAug-LeCam.yaml
StyleGAN2 matched with StyleGAN2-paper.yaml
StyleGAN2-DiffAug matched with StyleGAN2-256Res-DiffAug.yaml
StyleGAN2-ADA matched with StyleGAN2-256Res-ADA.yaml
StyleGAN2-APA matched with StyleGAN2-256Res-APA.yaml
StyleGAN2-LeCam matched with StyleGAN2-paper.yaml
StyleGAN2-DiffAug-LeCam matched with StyleGAN2-256Res-DiffAug-LeCam.yaml


In [36]:
# Check if evey excel column is matched with proper configurations and modify config if needed.
# runs["TACGAN"]["cfg"] = "ACGAN-Mod-TAC.yaml"
# runs["CRGAN"]["cfg"] = "BigGAN-CR.yaml"
# runs["ICRGAN"]["cfg"] = "BigGAN-ICR.yaml"
# runs["ADCGAN"]["cfg"] = "ACGAN-Mod-Big-ADC.yaml"
runs["BigGAN-LeCam"]["cfg"] = "BigGAN-256Res-LeCam.yaml"
runs["StyleGAN2-LeCam"]["cfg"] = "StyleGAN2-256Res-LeCam.yaml"

print("After fixing...")
for key, value in runs.items():
    print(f"{key} matched with {value['cfg']}")

After fixing...
BigGAN matched with BigGAN-256Res.yaml
BigGAN-DiffAug matched with BigGAN-256Res-DiffAug.yaml
BigGAN-ADA matched with BigGAN-256Res-ADA.yaml
BigGAN-APA matched with BigGAN-256Res-APA.yaml
BigGAN-LeCam matched with BigGAN-256Res-LeCam.yaml
BigGAN-DiffAug-LeCam matched with BigGAN-256Res-DiffAug-LeCam.yaml
StyleGAN2 matched with StyleGAN2-paper.yaml
StyleGAN2-DiffAug matched with StyleGAN2-256Res-DiffAug.yaml
StyleGAN2-ADA matched with StyleGAN2-256Res-ADA.yaml
StyleGAN2-APA matched with StyleGAN2-256Res-APA.yaml
StyleGAN2-LeCam matched with StyleGAN2-256Res-LeCam.yaml
StyleGAN2-DiffAug-LeCam matched with StyleGAN2-256Res-DiffAug-LeCam.yaml


In [40]:
# write script file
i = 1
with open("eval"+dataset+".sh", "w") as f:
    f.write("wandb offline\n")
    prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -metrics is fid prdc -best --seed 1234 -mpc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    if dataset == "ImageNet":
        prefix_ = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -metrics is fid prdc -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    elif dataset == "CIFAR10":
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer wo_resize --post_resizer tailored -std_stat -std_max 128 -std_step 128 -metrics is fid prdc -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    else:
        prefix_ = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -metrics is fid prdc -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    train = " -ref train"
    for (key, value) in runs.items():
        others1 = " -ckpt " + os.path.join(ckpt_path, value['run_1']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        if value['run_2'] != '':
            others2 = " -ckpt " + os.path.join(ckpt_path, value['run_2']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        if value['run_3'] != '':
            others3 = " -ckpt " + os.path.join(ckpt_path, value['run_3']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        if "Style" in key:
            prefix = prefix_style
        else:
            prefix = prefix_
        f.write(prefix+train+others1)
        f.write(f"echo =================={i}/{15} completed =============\n")
        i+=1 
        if value['run_2'] != '':
            f.write(prefix+train+others2)
            f.write(f"echo =================={i}/{15} completed =============\n")
            i+=1
        if value['run_3'] != '':
            f.write(prefix+train+others3)
            f.write(f"echo =================={i}/{15} completed =============\n")
            i+=1

In [ ]:
#### RUN 
#### bash eval_clean_CIFAR10.sh
#### bash eval....sh

In [ ]:
import pickle
with open("/root/code/joong_test/PyTorch-StudioGAN/eval_pickles/DCGAN-train-2022_01_11_20_32_07---clean.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
print(data)
print(float(data["IS"]))

In [ ]:
def fill_excel(col, start_row, data):
    col[start_row].value = float(data["IS"])
    col[start_row+4].value = data["FID"]
    col[start_row+8].value = data["Improved_Precision"]
    col[start_row+12].value = data["Improved_Recall"]
    col[start_row+16].value = data["Density"]
    col[start_row+20].value = data["Coverage"]
    
def modify_name(col, resize_fn):
    if col[6].value == "CIFAR10-BigGAN-Deep-train-2022_02_02_21_56_10":
        name1 = "BigGAN-Deep-StudioGAN-train-2022_02_02_21_56_10"+ "---"+resize_fn+".pickle"
        name2 = "BigGAN-Deep-StudioGAN-train-2022_02_02_21_56_49"+ "---"+resize_fn+".pickle"
        name3 = "BigGAN-Deep-StudioGAN-train-2022_02_02_22_01_49"+ "---"+resize_fn+".pickle"
    elif col[6].value[8:].startswith("BigGAN-Mod"):
        name1 = col[6].value[8:][0:6]+col[6].value[8:][10:]+ "---"+resize_fn+".pickle"
        name2 = col[7].value[8:][0:6]+col[7].value[8:][10:]+ "---"+resize_fn+".pickle"
        name3 = col[8].value[8:][0:6]+col[8].value[8:][10:]+ "---"+resize_fn+".pickle"
    else:
        name1 = col[6].value[8:] + "---"+resize_fn+".pickle"
        name2 = col[7].value[8:] + "---"+resize_fn+".pickle"
        name3 = col[8].value[8:] + "---"+resize_fn+".pickle"
    return name1, name2, name3


In [ ]:
# load Excel File and read informations
wb_clean = openpyxl.load_workbook("CIFAR10_clean.xlsx")
ws_clean = wb_clean['Sheet1']
pickles_path = "/root/code/joong_test/PyTorch-StudioGAN/eval_pickles/"
pickles_list = os.listdir(pickles_path)

all_columns = ws_clean.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        name1, name2, name3 = modify_name(col, "clean")
        assert name1 in pickles_list, name1 + " Pickle Does not exist"
        assert name2 in pickles_list, name2 + " Pickle Does not exist"
        assert name3 in pickles_list, name3 + " Pickle Does not exist"

        with open(os.path.join(pickles_path, name1), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 10, data)
        with open(os.path.join(pickles_path, name2), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 11, data)
        with open(os.path.join(pickles_path, name3), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 12, data)
wb_clean.save("CIFAR10_clean.xlsx")

In [ ]:
# load Excel File and read informations
wb_legacy = openpyxl.load_workbook("CIFAR10_legacy.xlsx")
ws_legacy = wb_legacy['Sheet1']
pickles_path = "/root/code/joong_test/PyTorch-StudioGAN/eval_pickles/"
pickles_list = os.listdir(pickles_path)

all_columns = ws_legacy.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        name1, name2, name3 = modify_name(col, "legacy")
        assert name1 in pickles_list, name1 + " Pickle Does not exist"
        assert name2 in pickles_list, name2 + " Pickle Does not exist"
        assert name3 in pickles_list, name3 + " Pickle Does not exist"

        with open(os.path.join(pickles_path, name1), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 10, data)
        with open(os.path.join(pickles_path, name2), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 11, data)
        with open(os.path.join(pickles_path, name3), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 12, data)
wb_legacy.save("CIFAR10_legacy.xlsx")

In [ ]:
with open("/root/code/joong_test/PyTorch-StudioGAN/eval_pickles/ACGAN-Mod-train-2022_01_11_21_42_44---legacy.pickle", "rb") as f:
    data = pickle.load(f)
data